In [37]:
import pathlib
import random
import string
import re
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

# parse data
with open('spa.txt', 'r', encoding='utf-8') as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    eng, spa = line.split("\t")[:-1]
    spa = "[start] " + spa + " [end]"
    text_pairs.append((eng, spa))


for _ in range(5):
    print(random.choice(text_pairs))

("You must do the work even if you don't like it.", '[start] Tienes que hacer el trabajo aunque no te guste. [end]')
("It'll just take a second.", '[start] Solo tomará un segundo. [end]')
('Most of the policemen lost their jobs.', '[start] La mayoría de los policías perdió su trabajo. [end]')
("He is precisely the man we're looking for.", '[start] Precisamente él es el hombre que buscamos. [end]')
('I need a bag. Will you lend me one?', '[start] Necesito una bolsa. ¿Me puedes prestar una? [end]')


In [38]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

138440 total pairs
96908 training pairs
20766 validation pairs
20766 test pairs


In [39]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size = 15000
sequence_length = 20
batch_size = 64


def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


eng_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
spa_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_eng_texts = [pair[0] for pair in train_pairs]
train_spa_texts = [pair[1] for pair in train_pairs]
eng_vectorization.adapt(train_eng_texts)
spa_vectorization.adapt(train_spa_texts)

In [40]:
def format_dataset(eng, spa):
  eng = eng_vectorization(eng)
  spa = spa_vectorization(spa)
  return ({
    "english": eng,
    "spanish": spa[:, :-1],
  }, spa[:, 1:])  

def make_dataset(pairs):
  eng_texts, spa_texts = zip(*pairs)
  eng_texts = list(eng_texts)
  spa_texts = list(spa_texts)
  dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
  dataset = dataset.batch(batch_size)
  dataset = dataset.map(format_dataset, num_parallel_calls=4)
  return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

for inputs, targets in train_ds.take(1):
  print(f"inputs['english'].shape: {inputs['english'].shape}")
  print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
  print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


In [42]:
from tensorflow import keras
from tensorflow.keras import layers

from keras.layers import Input, LSTM, Dense
from keras.models import Model

embed_dim = 256
latent_dim = 1024
# source = keras.Input(shape=(None,), dtype="int64", name="english")
# x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
# encoded_source = layers.Bidirectional(
# layers.GRU(latent_dim), merge_mode="sum")(x)

# past_target = keras.Input(shape=(None,), dtype="int64", name="spanish")
# x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
# decoder_gru = layers.GRU(latent_dim, return_sequences=True)
# x = decoder_gru(x, initial_state=encoded_source)
# x = layers.Dropout(0.5)(x)
# target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
# seq2seq_rnn = keras.Model([source, past_target], target_next_step)

# Encoder training setup
encoder_inputs = Input(shape=(None,),name = 'english')
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(x)
encoder_states = [state_hidden, state_cell]

# Decoder training setup:
decoder_inputs = Input(shape=(None,),name = 'spanish')
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(x, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Building the training model:
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


# seq2seq_rnn.compile(
# optimizer="rmsprop",
# loss="sparse_categorical_crossentropy",
# metrics=["accuracy"])
# seq2seq_rnn.fit(train_ds, epochs=15, validation_data=val_ds)

training_model.compile(
optimizer="rmsprop",
loss="sparse_categorical_crossentropy",
metrics=["accuracy"])
training_model.fit(train_ds, epochs=15, validation_data=val_ds)


Epoch 1/15
1515/1515 [==============================] - 143s 89ms/step - loss: 1.5928 - accuracy: 0.3973 - val_loss: 1.2764 - val_accuracy: 0.4864
Epoch 2/15
1515/1515 [==============================] - 133s 88ms/step - loss: 1.1585 - accuracy: 0.5333 - val_loss: 1.0519 - val_accuracy: 0.5687
Epoch 3/15
1515/1515 [==============================] - 133s 88ms/step - loss: 0.9602 - accuracy: 0.6065 - val_loss: 0.9599 - val_accuracy: 0.6115
Epoch 4/15
1515/1515 [==============================] - 133s 88ms/step - loss: 0.8241 - accuracy: 0.6588 - val_loss: 0.9216 - val_accuracy: 0.6295
Epoch 5/15
1515/1515 [==============================] - 133s 88ms/step - loss: 0.7222 - accuracy: 0.6986 - val_loss: 0.9153 - val_accuracy: 0.6354
Epoch 6/15
1515/1515 [==============================] - 133s 88ms/step - loss: 0.6458 - accuracy: 0.7313 - val_loss: 0.9237 - val_accuracy: 0.6366
Epoch 7/15
1515/1515 [==============================] - 134s 88ms/step - loss: 0.5824 - accuracy: 0.7564 - val_loss: 0

In [43]:
import numpy as np
spa_vocab = spa_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
  tokenized_input_sentence = eng_vectorization([input_sentence])
  decoded_sentence = "[start]"
  for i in range(max_decoded_sentence_length):
    tokenized_target_sentence = spa_vectorization([decoded_sentence])
    # next_token_predictions = seq2seq_rnn.predict(
    next_token_predictions = training_model.predict(
    [tokenized_input_sentence, tokenized_target_sentence])
    sampled_token_index = np.argmax(next_token_predictions[0, i, :])
    sampled_token = spa_index_lookup[sampled_token_index]
    decoded_sentence += " " + sampled_token
    if sampled_token == "[end]":
      break
  return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
  input_sentence = random.choice(test_eng_texts)
  print("-")
  print(input_sentence)
  print(decode_sequence(input_sentence))

-
I'll call you as soon as I'm free.
[start] te llamaré cuando quieras [end]
-
His jokes had us in stitches.
[start] sus palabras nos [UNK] a la pegó en el correo [end]
-
Tom couldn't think of any reason for not giving Mary the present.
[start] tom no podía pensar en ningún motivo para no [UNK] a mary [end]
-
A sponge absorbs water.
[start] una frente está agua [end]
-
If anyone comes to see me, tell him that I am out.
[start] si alguien llega mientras necesito estado mirando [end]
-
I'm getting married on Monday.
[start] me estoy de lunes [end]
-
What are you talking about?
[start] de qué estás hablando [end]
-
Have you lost weight?
[start] has perdido peso [end]
-
He won the election by a large majority.
[start] Él ganó la eso más grande de una gran [end]
-
There's your friend.
[start] ahí es tu amigo [end]
-
It's improved.
[start] está [UNK] [end]
-
I'm almost certain about that.
[start] casi estoy de eso al respecto [end]
-
My spirit animal is a rabbit. What's yours?
[start] mi pro

In [45]:
print(decode_sequence('What do you want to eat?'))
print(decode_sequence('How was your day?'))
print(decode_sequence('I am going to the beast whith my best friend Ivan'))


[start] qué quiere comer [end]
[start] cómo estuvo tu día [end]
[start] voy a ir a la del nuevo amigo verdad [end]
